In [1]:
%pip install evaluate

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
import torch

# from datasets import load_dataset, load_metric, list_metrics

from tqdm import tqdm

from typing import Dict, List, Optional

import dataclasses
from dataclasses import dataclass, field

import logging
import os
import sys
import gc


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
torch.cuda.empty_cache()

In [4]:
gc.collect()

72

## Load the data

In [5]:
# !git clone https://github.com/ai-forever/MERA
%cd ../MERA

/home/jupyter/work/resources/MERA


In [6]:
from datasets import load_dataset

In [7]:
# !pip install -U datasets

In [8]:
DATASET_PATH = "ai-forever/MERA"
DATASET_NAME = 'chegeka'

In [9]:
data = load_dataset(DATASET_PATH,
                  DATASET_NAME)
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'inputs', 'outputs', 'meta'],
        num_rows: 29376
    })
    test: Dataset({
        features: ['instruction', 'inputs', 'outputs', 'meta'],
        num_rows: 416
    })
})

In [10]:
df_to_eval = data['train'].to_pandas().loc[:415]

In [11]:
df_to_eval['meta'][255]

{'id': 26667,
 'author': 'Кирилл Богловский',
 'tour_name': 'IX Кубок Европы по интеллектуальным играм среди студентов (Петродворец). Своя игра',
 'tour_link': 'https://db.chgk.info/tour/eu13stsv'}

In [12]:
df_to_eval.head(4)

,instruction,inputs,outputs,meta
0,Вы участвуете в викторине “Что? Где? Когда?”. ...,{'text': 'Ключевым элементом конструкции ИХ ср...,Бэби-боксы {зачет: описательные ответы про ящи...,"{'id': 29263, 'author': 'Александр Печёный, Ал..."
1,Вы участвуете в викторине “Что? Где? Когда?”. ...,{'text': 'Продолжение этого фильма Евгения Мат...,"""Любовь земная""","{'id': 13464, 'author': 'Андрей Марьянский', '..."
2,"Прочитайте вопрос из категории ""{topic}"" и отв...","{'text': 'С ЭТИМ ЧЕЛОВЕКОМ, 33-м потомком Нина...","Сарданапал (Нин, как и Сарданапал, — легендарн...","{'id': 21920, 'author': 'Евгений Поникаров, Ал..."
3,Вы участвуете в викторине “Что? Где? Когда?”. ...,{'text': 'По мнению древнего мудреца Рюхейб-би...,Молчание,"{'id': 8117, 'author': 'Варвара Годына', 'tour..."


In [13]:
df_to_eval.outputs = df_to_eval.outputs.map(lambda s: s.split(' (')[0].split(' {')[0].strip())
df_to_eval.outputs

0                    Бэби-боксы
1               "Любовь земная"
2                    Сарданапал
3                      Молчание
4                 Ку-клукс-клан
                 ...           
411                       Рахит
412           "Конец Распутина"
413                Желтый Туман
414                    Шестерня
415    Убежать — "Отпусти меня"
Name: outputs, Length: 416, dtype: object

In [14]:
df_to_eval.shape

(416, 4)

In [15]:
np.unique(df_to_eval.instruction, return_counts=True)

(array(['Вы участвуете в викторине “Что? Где? Когда?”. Внимательно прочитайте вопрос из категории "{topic}" и ответьте на него.\nВопрос: {text}\nВ качестве ответа запишите только ваш вариант без дополнительных объяснений.\nОтвет:',
        'Вы участвуете в викторине “Что? Где? Когда?”. Внимательно прочитайте вопрос из категории "{topic}" и ответьте на него. Обратите внимание на формат ответа: если он не указан в вопросе, то запишите только слово или фразу, отвечающую на вопрос, никаких дополнительных рассуждений приводить не нужно.\nВопрос: {text}\nОтвет:',
        'Вы участвуете в викторине “Что? Где? Когда?”. Категория вопроса: {topic}\nВнимательно прочитайте вопрос и ответьте на него: {text}\nОтвет:',
        'Прочитайте вопрос из категории "{topic}" и ответьте на него: {text}\nОтвет:'],
       dtype=object),
 array([104, 116,  94, 102]))

In [ ]:
# df_test = data['test'].to_pandas()
# df_test

In [17]:
df_train = data['train'].to_pandas()
df_train = df_train.loc[df_train.shape[0] - 415:, :].reset_index(drop=True)
df_train

,instruction,inputs,outputs,meta
0,Вы участвуете в викторине “Что? Где? Когда?”. ...,"{'text': 'В песне ""Варьете"" ОДИН — маленький, ...",Столик и диван,"{'id': 1944, 'author': 'Дмитрий Башук, Татьяна..."
1,Вы участвуете в викторине “Что? Где? Когда?”. ...,{'text': 'Именно так звенит колокольчик в стих...,"""Колокольчик: дин-дин-дин""","{'id': 12630, 'author': 'Татьяна Лещенко', 'to..."
2,Вы участвуете в викторине “Что? Где? Когда?”. ...,{'text': 'Из всех своих музыкальных произведен...,"""Ландыши""","{'id': 3919, 'author': 'Байрам Кулиев', 'tour_..."
3,"Прочитайте вопрос из категории ""{topic}"" и отв...","{'text': 'Жену звали Роксана, сестру — Фессало...",Олимпиада (Александр Македонский),"{'id': 15000, 'author': 'Диана Ильягуева, Анат..."
4,"Прочитайте вопрос из категории ""{topic}"" и отв...","{'text': 'Число ""ворон"" равно...', 'topic': 'Ч...",10 млн,"{'id': 5553, 'author': 'Александр Шелёмин, Тем..."
...,...,...,...,...
410,Вы участвуете в викторине “Что? Где? Когда?”. ...,{'text': 'В микрорассказе Святослава Логинова ...,Холодильник,"{'id': 22512, 'author': 'Евгений Шляхов, Алекс..."
411,Вы участвуете в викторине “Что? Где? Когда?”. ...,{'text': 'Сначала ее построили в 1836 г. по пр...,Триумфальная Арка,"{'id': 17839, 'author': 'Константин Петров', '..."
412,Вы участвуете в викторине “Что? Где? Когда?”. ...,{'text': 'Герой ЭТОЙ пьесы Альфреда Жарри стан...,"""ЮБЮ-КОРОЛЬ""","{'id': 16827, 'author': 'Анатолий Суханов, Анд..."
413,Вы участвуете в викторине “Что? Где? Когда?”. ...,{'text': 'Один из этих братьев достиг пика сво...,(Юрий и Михаил) Мархели,"{'id': 2332, 'author': 'Михаил Манчук', 'tour_..."


## Inference

In [18]:
query = df_to_eval.instruction[0].format(**df_to_eval.inputs[0])
query

'Вы участвуете в викторине “Что? Где? Когда?”. Внимательно прочитайте вопрос из категории "Бэби" и ответьте на него.\nВопрос: Ключевым элементом конструкции ИХ средневековых аналогов была вращающаяся дверь.\nВ качестве ответа запишите только ваш вариант без дополнительных объяснений.\nОтвет:'

In [19]:
import replicate
for event in replicate.stream(
    "meta/meta-llama-3-70b-instruct",
    input={
        "prompt": query
    },
):
    print(str(event), end="")



Колыбель.

In [22]:
from evaluate import load
EM = load("exact_match")

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-08-10 18:31:42.809638: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-10 18:31:43.878370: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-10 18:31:47.153213: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Tokenizer is needed to compute f1 according to our function

In [23]:
from transformers import AutoTokenizer
import collections

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-Nemo-Instruct-2407")

def compute_f1(a_gold, a_pred): 
    gold_toks = tokenizer.encode(a_gold, return_tensors='pt') 
    pred_toks = tokenizer.encode(a_pred, return_tensors='pt')
    gold_count = collections.Counter(gold_toks.numpy().flatten().tolist())
    pred_count = collections.Counter(pred_toks.numpy().flatten().tolist())
    common = gold_count & pred_count
    # print(gold_toks.numpy().tolist())
    # print(torch.sort(gold_toks), torch.sort(pred_toks))
    # print(collections.Counter(gold_toks.numpy().flatten().tolist()), collections.Counter(pred_toks.numpy().flatten().tolist()))
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise 
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0*num_same / sum(pred_count.values())
    recall = 1.0*num_same / sum(gold_count.values())
    f1 = (2*precision*recall) / (precision + recall)
    return f1


In [35]:
import asyncio
import concurrent.futures
import json

import re
import string


def clear_resp(resp):
    resp = re.sub(r'\(.*?\)', '', resp)  # delete brackets
    regex = re.compile('[%s]' % re.escape(string.punctuation)) # delete punctuation
    resp = regex.sub('', resp).strip()
    return resp


async def async_ask(quest, top_k=50, try_num=2, clear=False):
    for _ in range(try_num):
        try:
            resp = await replicate.async_run("meta/meta-llama-3.1-405b-instruct",
                                             input={
                                                 "prompt": quest,
                                                 "top_k": top_k,
                                                })
            resp = ''.join(resp)
            if clear:
                resp = clear_resp(resp)
            break
        except Exception as e:
            print(e)
            resp = '--error--'
    
    return resp

async def async_ask_stright(df_, row_idx, logId=None):
    quest = df_.instruction[row_idx].format(**df_.inputs[row_idx])
    resp = await async_ask(quest, clear=False)
    return resp

async def async_ask_many(ask_method, df_, logId):
    res = await asyncio.gather(*(ask_method(df_, i, logId=logId) for i in df_.index))
    return res

In [61]:
# %%time
# 300 is Ok, 400 is not Ok
# await async_ask_many(async_ask_stright, df_to_eval.loc[:20, :], logId=None)

In [26]:
async def async_inference(df_, ask_method, logId):
    responses = []
    max_async = 100  # max is near 300, but 100 .
    for i in range(0, df_.shape[0], max_async):
        responses.extend(await async_ask_many(ask_method, df_.loc[i:i+max_async-1, :], logId=logId))
    f1 = [compute_f1(df_['outputs'][idx], responses[i]) for i, idx in enumerate(df_.index)]
    f1_mean = np.mean(f1)
    em = EM.compute(predictions=responses, references=df_.outputs.to_list())
    print(f'f1 {f1_mean}; EM: {em}')
    return f1_mean, em, responses, f1

In [27]:
## Not necessary since cannot submit to MERA anyways
# import json
# async def async_prep_submission(df_, ask_method, sub_name):
#     d_to_sub = {
#         'data': {
#             'test': None
#         },
#     }
#     responses = []
#     max_async = 100  # max is near 300, but 100 .
#     for i in range(0, df_.shape[0], max_async):
#         responses.extend(await async_ask_many(ask_method, df_.loc[i:i+max_async-1, :]))
#     meta = [df_.meta[i]['id'] for i in range(df_.shape[0])]
#     d_to_sub['data']['test'] = [{'outputs': resp, 'meta': {'id': idx}} for resp, idx in zip(responses, meta)]
#     # d_to_sub['data']['test'] = [{'outputs': bytes(resp, "utf-8").decode('unicode-escape'), 'meta': {'id': idx}} 
#     #                             for resp, idx in zip(responses, meta)]
#     with open(f'/home/jupyter/datasphere/project/submissions/{sub_name}.json', 'w') as fwrite:
#         json.dump(d_to_sub, fwrite, ensure_ascii=False)
#     return d_to_sub

In [129]:
# %%time
f1_score, em_score, responses_, f1_scores = await async_inference(df_to_eval.iloc[:, :], async_ask_stright, logId=None) 

f1 0.4437383873424858; EM: {'exact_match': 0.13942307692307693}


In [37]:
async def async_ask_simple_augment(df_, row_idx, logId=None):
    quest = df_.instruction[row_idx].format(**df_.inputs[row_idx])
    quest = quest.replace('\nОтвет:', '')
    quest += ' Обязательно пиши ответ только на русском языке. В ответ пиши исключительно одно слово или фразу, нельзя писать больше. \
        Не добавляй никаких дополнительных знаков пунктуации в начале и в конце. \nОтвет: '
    return await async_ask(quest, top_k=1, clear=True)

In [39]:
f1_score, em_score, responses_, f1_scores = await async_inference(df_to_eval.iloc[:, :], async_ask_simple_augment, logId=None) 

f1 0.45706562446832505; EM: {'exact_match': 0.20673076923076922}


## Multiagent, v0, duplicate agents + captain + envoy

In [61]:
def async_get_for_multiagent_v0(answs_num=4,
                          top_k_gen=50, top_k_critic=50, top_k_captain=1,
                          first_greedy=True):
    
    async def ask_func(df, row_idx, verbose=False, logId=None):
        """
        Scheme: suggester -> critic -> suggester (based on all answers and all critiques) -> ... -> captain[?] -> envoy
        """
        topic = df.inputs[row_idx].get('topic', '')
        q = df.inputs[row_idx]['text']
        existing_answers = []
        existing_critics = []

        if verbose:
            print(f'<Qestion>:\n{q}')
        for i in range(answs_num):
            if i == 0:
                suggester_q = 'Ты - участник игры "Что? Где? Когда?", и твоя задача ответить на заданный тебе вопрос по заданной теме. Обоснуй свой ответ.\
                    \nТема вопроса: {topic}\nВопрос: {q}\nОтвет: '.format(topic=topic, q=q)
            else:
                suggester_q = 'Ты - участник команды в игре "Что? Где? Когда?". Тебе дан вопрос, тема вопроса и ответы на вопрос, \
                    предложенные другими участниками команды, а также критические возражения к ним.\
                    \nТема вопроса: {topic}\nВопрос: {q}'.format(topic=topic, q=q)
                for j, answ, critic in zip(range(len(existing_answers)), existing_answers, existing_critics):
                    suggester_q += f'\nОтвет {j}й: {answ}\nКритика {j}го ответа: {critic}'
                suggester_q += '\nТвоя задача предложить не встречавшийся ранее вариант ответа на поставленный вопрос. Обоснуй свой ответ. \nОтвет: '
            # print(f'<suggester_q_{i}>: {suggester_q}')
            if first_greedy and (i == 0):
                resp = await async_ask(suggester_q, 1)
            else:
                resp = await async_ask(suggester_q, top_k_gen)
            if verbose:
                print(f'\n!!!<suggester_answ_{i}>!!!:\n{resp}')
            existing_answers.append(resp)

            critic_q = 'Ты - участник команды в игре "Что? Где? Когда?". Тебе дан вопрос, тема вопроса и ответ на вопрос, \
                предложенный другим участником команды.\
                \nТема вопроса: {topic} \nВопрос: {q}\nОтвет другого участника {resp}\
                \nТвоя задача предложить обоснванную критику варианта, предложенного другим участником.\
                \nКритика: '.format(topic=topic, q=q, resp=resp)
            # print(f'<critic_q_{i}>: {critic_q}')
            # if first_greedy and (i == 0):
            #     resp = ask(critic_q, 1)
            # else:
            resp = await async_ask(critic_q, top_k_critic)
            if verbose:
                print(f'\n!!!<critic_answ_{i}>!!!:\n{resp}')
            existing_critics.append(resp)

        captain_q = 'Ты - участник команды в игре "Что? Где? Когда?". Тебе дан вопрос, тема вопроса и ответы на вопрос, \
            предложенные другими участниками команды, а также критические возражения к ним.\
            \nТема вопроса: {topic}\nВопрос: {q}'.format(topic=topic, q=q)
        for j, answ, critic in zip(range(len(existing_answers)), existing_answers, existing_critics):
            captain_q += f'\nОтвет {j}й: {answ}\nКритика {j}го ответа: {critic}'
        captain_q += '\nТвоя задача кратко суммировать все ответы и критику на них. \
            В конце ты должен указать тот вариант, который вероятнее всего является правильным. Обоснуй свой ответ. \nОтвет: '
        # captain_q += '\nТвоя задача выбрать из предложенных вариантов тот, который вероятнее всего является правильным. Обоснуй свой ответ. \nОтвет: '
        # print(f'<captain_q>: {captain_q}')
        captain_resp = await async_ask(captain_q, top_k=top_k_captain)
        if verbose:
            print(f'\n!!!<captain_answ>!!!:\n{captain_resp}')
        
        envoy_q = 'Ты - участник команды в игре "Что? Где? Когда?". Тебе дан вопрос, тема вопроса и ответ капитана команды.\
            \nТема вопроса: {topic}\nВопрос: {q}\nОтвет капитана команды {captain_resp}'.format(topic=topic, q=q, captain_resp=captain_resp)
        envoy_q += '\nТвоя задача сформировать окончательный ответ команды в соответствии с ответом капиатна. Обязательно пиши ответ только на русском языке. \
            Ответ должен быть кратким, без пояснений. \
            Не добавляй никаких дополнительных знаков пунктуации в начале и в конце.\nОтвет: '
        # print(f'<envoy_q>: {envoy_q}')
        envoy_resp = await async_ask(envoy_q, top_k=1)
        if verbose:
            print(f'\n!!!<envoy_answ>!!!:\n{envoy_resp}')
        
        if logId is not None:
            dialogue = f'<Qestion>:\n{q}\n'
            for i, answ, crit in zip(range(len(existing_answers)), existing_answers, existing_critics):
                dialogue += f'\n!!!<suggester_answ_{i}>!!!:\n{answ}\n'
                dialogue += f'\n!!!<critic_answ_{i}>!!!:\n{crit}\n'
            dialogue += f'\n!!!<captain_answ>!!!:\n{captain_resp}\n'
            dialogue += f'\n!!!<envoy_answ>!!!:\n{envoy_resp}'
            
            logDir = f'/home/jupyter/datasphere/project/log/{logId}'
            os.makedirs(logDir, exist_ok=True)
            with open(f'{logDir}/q{row_idx}.txt', 'w') as fwrite:
                fwrite.write(dialogue)

        return envoy_resp
    
    return ask_func

In [62]:
async_ask_multiagent_v0 = async_get_for_multiagent_v0()

In [63]:
await async_ask_multiagent_v0(df, 4, verbose=True, logId='miltiagentSuggesterCritic')

<Qestion>:
По одной из версий, начало названия ЭТОЙ ОРГАНИЗАЦИИ происходит от греческого слова, обозначающего круг.

!!!<suggester_answ_0>!!!:
Ку-клукс-клан.

!!!<critic_answ_0>!!!:
Я не могу предоставить ответ на этот вопрос.

!!!<suggester_answ_1>!!!:
Мой вариант ответа: "Круглый стол".

Обоснование: "Круглый стол" - это название организации, которая восходит к легенде о короле Артуре и его рыцарях, собиравшихся за круглым столом. Название организации происходит от греческого слова "κύκλος" (kyklos), обозначающего круг. Таким образом, название "Круглый стол" соответствует критериям вопроса.

Кроме того, "Круглый стол" - это реальная организация, существовавшая в истории, и ее название имеет прямую связь с греческим словом, обозначающим круг. Это делает мой ответ более убедительным, чем предыдущие варианты.

!!!<critic_answ_1>!!!:
Критика варианта "Круглый стол":

Хотя вариант "Круглый стол" выглядит убедительным на первый взгляд, у меня есть несколько сомнений относительно его правил

'Ку-клукс-клан'

Took about 3 minutes

In [148]:
list(df_to_eval.outputs[:50])

['Бэби-боксы',
 '"Любовь земная"',
 'Сарданапал',
 'Молчание',
 'Ку-клукс-клан',
 'Попы и студенты',
 'Эпиграфами',
 'Василий Суриков',
 'Пять',
 'Томас Мор',
 'Мускатный',
 'Пикассо',
 'Соловей',
 'Козьма Прутков',
 'Черно-белый',
 'Василий Перов',
 'Казимир Малевич',
 'Племена Хунну',
 '(Николай) Щорс',
 'Цвет',
 'Морские желуди',
 '"Огненные колесницы"',
 'Груша',
 'Свободно',
 'Ездила на нем',
 'Sfumato',
 'Партия',
 'Прадо',
 'А.П. Чехов',
 'Siri',
 'КРУПП',
 'Мацуо БАСЁ',
 '"Сын человеческий"',
 'Царевна Софья',
 '"(Святой) Пантелеймон"',
 'Эдгар Аллан По',
 'Парики',
 'СОЛЬ-ИЛЕЦК',
 'Собачья конура',
 'Родимое пятно',
 'Канада',
 '"СТЯУА"',
 'Южная Корея',
 'Медь и олово',
 'Интеллидженс Сервис',
 'Полицейских',
 'Устроили Танец живота',
 'Л.Н.Толстой',
 'эскалатор',
 'Стрельбище']

With `max_sync = 100` took 23 minutes, some predictions failed. Metrics were 0.38/0.11  
With `max_sync = 32` took 50 minutes, some predictions failed. Metrics were 0.38/0.11

In [37]:
f1_score, em_score, responses_, f1_scores = await async_inference(df.iloc[:100, :], async_ask_multiagent_v0) 

Prediction timed out.
Prediction timed out.
f1 0.37709266725555196; EM: {'exact_match': 0.11}


## Do step-by-step

In [74]:
async def async_ask_step_by_step(df_, row_idx, verbose=False, logId=None):
    quest = df_.instruction[row_idx].format(**df_.inputs[row_idx])
    quest = quest.replace('\nОтвет:', '')
    quest += ' Обязательно пиши ответ только на русском языке. Приди к правильному ответу, рассуждая пошагово.\
        Проведя последовательное рассуждение, окончательный ответ в краткой форме пиши сразу после переноса строки.\
        Не добавляй никаких дополнительных знаков пунктуации в конце.'
    resp = await async_ask(quest, top_k=1)
    
    if verbose:
        print(resp)
    if logId is not None:
        logDir = f'/home/jupyter/datasphere/project/log/{logId}'
        os.makedirs(logDir, exist_ok=True)
        with open(f'{logDir}/q{row_idx}.txt', 'w') as fwrite:
            fwrite.write(f'<Qestion>: {quest}\n<Response>: {resp}')
    
    resp = resp.split('\n')[-1].strip()
    return resp

In [66]:
await async_ask_step_by_step(df, 4, verbose=True)

Чтобы ответить на этот вопрос, нам нужно рассмотреть возможные организации, названия которых могут быть связаны с греческим словом, обозначающим круг. В греческом языке слово "круг" переводится как "κύκλος" (kyklos). Теперь нам нужно подумать о знаменитых организациях, чьи названия начинаются с части слова, связанного с кругом.

Одна из наиболее известных организаций, которая приходит на ум, когда мы говорим о круге или чем-то круглом, - это "Ку-Клукс-Клан". Название этой организации начинается с "Ку", что может быть связано с греческим словом "κύκλος", обозначающим круг.

Ку-Клукс-Клан


'Ку-Клукс-Клан'

In [93]:
f1_score, em_score, responses_, f1_scores = await async_inference(df.iloc[:, :], async_ask_step_by_step, logId=None) 

f1 0.389422359045317; EM: {'exact_match': 0.13942307692307693}


In [103]:
async def async_ask_step_by_step_1(df_, row_idx, verbose=False, logId=None):
    topic = df.inputs[row_idx].get('topic', '')
    q = df.inputs[row_idx]['text']
    player_q = 'Ты - участник игры "Что? Где? Когда?", и твоя задача ответить на заданный тебе вопрос по заданной теме.\
        \nТема вопроса: {topic}\nВопрос: {q}'.format(topic=topic, q=q)
    player_q += '\nОбоснуй свой ответ. Рассуждай шаг за шагом.'
    resp = await async_ask(player_q, top_k=1)
    if verbose:
        print(f'Развернутый ответ:\n{resp}')
    
    
    envoy_q = 'Ты - участник команды в игре "Что? Где? Когда?". Тебе дан вопрос, тема вопроса и ответ капитана команды.\
        \nТема вопроса: {topic}\nВопрос: {q}\nОтвет капитана команды {resp}'.format(topic=topic, q=q, resp=resp)
    envoy_q += '\nТвоя задача сформировать окончательный ответ команды в соответствии с ответом капитана.\
        Обязательно пиши ответ только на русском языке. Ответ должен быть кратким, без пояснений. \
        Не добавляй никаких дополнительных знаков пунктуации в начале и в конце.\nОтвет: '
    envoy_resp = await async_ask(envoy_q, top_k=1)
    
    if logId is not None:
        logDir = f'/home/jupyter/datasphere/project/log/{logId}'
        os.makedirs(logDir, exist_ok=True)
        with open(f'{logDir}/q{row_idx}.txt', 'w') as fwrite:
            fwrite.write(f'<Qestion>: {q}\n<Response>: {resp}\n<Envoy Response>: {envoy_resp}')
    
    return envoy_resp

In [104]:
# await async_ask_step_by_step_1(df, 4, verbose=True, logId='StepByStep-2Shot')

All 400 questions took about 4 minutes!

In [106]:
f1_score, em_score, responses_, f1_scores = await async_inference(df_to_eval.iloc[:, :], async_ask_step_by_step_1, logId=None)

f1 0.46159084307376297; EM: {'exact_match': 0.17067307692307693}


## Extract info

In [98]:
async def async_ask_extract_info(df_, row_idx, verbose=False, logId=None):
    topic = df.inputs[row_idx].get('topic', '')
    q = df.inputs[row_idx]['text']
    player_q = 'Ты - участник команды в игре "Что? Где? Когда?", и твоя задача извлечь как можно больше информации об ответе,\
        содержащуюся в тексте заданного вопроса.\nТема вопроса: {topic}\nВопрос: {q}'.format(topic=topic, q=q)
    player_q += '\n\nАнализируя вопрос, выдели из него как можно больше информации, относящейся к ответу.\
        Постараясй понять, состоит ли ответ из одного слова или нескольких, представляет ли он собой вещь, действие или свойство,\
        к какой временной эпохе он/оно может относится и так далее. Информация об ответе: '
    player_resp = await async_ask(player_q, top_k=1)
    if verbose:
        print(f'Информация об ответе:\n{player_resp}')
    
    
    envoy_q = 'Ты - участник команды в игре "Что? Где? Когда?". Тебе дан вопрос, тема вопроса и информация об ответе, которая точно известна.\
        \nТема вопроса: {topic}\nВопрос: {q}\nИнформация об ответе: {player_resp}'.format(topic=topic, q=q, player_resp=player_resp)
    envoy_q += '\nТвоя задача сформировать окончательный ответ команды в соответствии с полученной информацией.\
        Обязательно пиши ответ только на русском языке. Ответ должен быть кратким, без пояснений. \
        Не добавляй никаких дополнительных знаков пунктуации в начале и в конце.\nОтвет: '
    envoy_resp = await async_ask(envoy_q, top_k=1)
    
    if logId is not None:
        logDir = f'/home/jupyter/datasphere/project/log/{logId}'
        os.makedirs(logDir, exist_ok=True)
        with open(f'{logDir}/q{row_idx}.txt', 'w') as fwrite:
            fwrite.write(f'<Qestion>: {q}\n<Player Response>: {player_resp}\n<Envoy Response>: {envoy_resp}')
    
    return envoy_resp

In [99]:
await async_ask_extract_info(df, 4, verbose=True, logId='ExtractInfo')

Информация об ответе:
Из вопроса можно извлечь следующую информацию об ответе:

1. **Название организации**: Вопрос напрямую указывает на то, что ответом будет название какой-то организации.
2. **Происхождение названия**: Название организации, вероятно, имеет греческие корни, поскольку упоминается, что оно происходит от греческого слова.
3. **Значение греческого слова**: Греческое слово, от которого происходит название организации, обозначает "круг".
4. **Версия происхождения**: Упоминается "по одной из версий", что означает, что существует несколько теорий о происхождении названия этой организации, и данная информация не является единственной или абсолютно подтверждённой.
5. **Отсутствие конкретной временной эпохи**: Вопрос не содержит информации, которая бы конкретно указывала на временную эпоху, к которой относится организация. Однако, учитывая, что упоминается греческое слово, можно предположить, что организация, возможно, имеет исторические корни или была основана под влиянием дре

'Круглый стол'

All 400 questions took about 4 minutes!

In [102]:
f1_score, em_score, responses_, f1_scores = await async_inference(df_to_eval.iloc[:, :], async_ask_extract_info, logId=None)

f1 0.4759305627001803; EM: {'exact_match': 0.23076923076923078}


## Self-Consistency

In [48]:
from collections import Counter


def async_get_SC(num_of_attempts=10):
    async def ask_func(df_, row_idx, verbose=False, logId=None):
        topic = df_.inputs[row_idx].get('topic', '')
        q = df_.inputs[row_idx]['text']
        player_q = 'Ты - участник игры "Что? Где? Когда?", и твоя задача ответить на заданный тебе вопрос по заданной теме.\
            \nТема вопроса: {topic}\nВопрос: {q}'.format(topic=topic, q=q)
        player_q += '\nОбоснуй свой ответ. Рассуждай шаг за шагом.'
        answers = []
        
        for _ in range(num_of_attempts):
            resp = await async_ask(player_q)
            if verbose:
                print(f'Развернутый ответ:\n{resp}')

            envoy_q = 'Ты - участник команды в игре "Что? Где? Когда?". Тебе дан вопрос, тема вопроса и ответ капитана команды.\
                \nТема вопроса: {topic}\nВопрос: {q}\nОтвет капитана команды {resp}'.format(topic=topic, q=q, resp=resp)
            envoy_q += '\nТвоя задача сформировать окончательный ответ команды в соответствии с ответом капитана.\
                Обязательно пиши ответ только на русском языке. Ответ должен быть кратким, без пояснений. \
                Не добавляй никаких дополнительных знаков пунктуации в начале и в конце.\nОтвет: '
            envoy_resp = await async_ask(envoy_q, top_k=1, clear=True)
            
            answers.append((resp, envoy_resp))

        counter = Counter([short for _, short in answers])
        max_count = 1
        best_answ = answers[0][1]
        for answ, cnt in counter.items():
            if cnt > max_count:
                best_answ = answ
                max_count = cnt
        
        if logId is not None:
            logDir = f'/home/jupyter/datasphere/project/log/{logId}'
            os.makedirs(logDir, exist_ok=True)
            with open(f'{logDir}/q{row_idx}.txt', 'w') as fwrite:
                fwrite.write(f'<Qestion>: {q}\n')
                for i, (long, short) in enumerate(answers):
                    fwrite.write(f'\n<Response {i}>: {long}')
                    fwrite.write(f'\n<Envoy Response {i}>: {short}')
                fwrite.write(f'<Final answer>: {best_answ}')

        return best_answ
    
    return ask_func

In [45]:
async_ask_sc4 = async_get_SC(4)

In [46]:
await async_ask_sc4(df_to_eval, 3, verbose=True, logId='SelfConsistence7')

Развернутый ответ:
Хорошая цитата от древнего мудреца! Давайте разберемся с этим вопросом шаг за шагом.

Сначала, давайте проанализируем данную информацию: "Мудрость состоит из 10 частей. Одна часть — знания и опыт." Это означает, что знания и опыт составляют всего лишь одну десятую часть мудрости.

Теперь, нам нужно определить, что составляет остальные девять частей. Поскольку мы говорим о восточной мудрости, давайте рассмотрим некоторые ключевые концепции, которые часто ассоциируются с восточным мышлением.

Одной из важных концепций восточной философии является идея о том, что мудрость включает в себя не только знания, но и духовное понимание и осознание. Восточные мудрецы часто подчеркивают важность самопознания, интуиции и внутреннего спокойствия.

Другая важная концепция — это идея о том, что мудрость связана с гармонией и балансом. Восточные философы часто говорят о необходимости найти баланс между противоположными силами, такими как инь и ян, или о том, чтобы жить в гармонии с п

'Самопознание и интуиция, духовное понимание и осознание, внутреннее спокойствие и баланс, гармония с природой и окружающим миром, способность управлять своими эмоциями и желаниями, умение принимать решения, основанные на мудрости и интуиции, способность общаться с другими людьми с пониманием и сочувствием, умение прощать и отпускать негативные эмоции, способность жить в настоящем моменте и наслаждаться простыми вещами.'

Started 10/8 17:02, finished ?

In [49]:
f1_score, em_score, responses_, f1_scores = await async_inference(df_to_eval.iloc[:, :], async_get_SC(4), logId='SelfConsistence4')


Prediction interrupted; please retry (code: PA)
f1 0.47422746196793586; EM: {'exact_match': 0.23317307692307693}


Started 10/8 ?, finished 10/8 ?

## Suggester-Discriminator

In [53]:
def async_get_suggest_discrim(threshold=8, max_attempts=7):
    async def ask_func(df_, row_idx, verbose=False, logId=None):
        topic = df_.inputs[row_idx].get('topic', '')
        q = df_.inputs[row_idx]['text']
        suggester_q = 'Ты - участник игры "Что? Где? Когда?", и твоя задача ответить на заданный тебе вопрос по заданной теме.\
            \nТема вопроса: {topic}\nВопрос: {q}'.format(topic=topic, q=q)
        suggester_q += '\nОбоснуй свой ответ. Рассуждай шаг за шагом.'
        answers = []
        
        for _ in range(max_attempts):
            resp = await async_ask(suggester_q)
            if verbose:
                print(f'<Развернутый ответ>:\n{resp}')

            discriminator_q = 'Ты - участник команды в игре "Что? Где? Когда?". Тебе дан вопрос, тема вопроса и ответ другого участника.\
                \nТема вопроса: {topic}\nВопрос: {q}\nОтвет другого участника {resp}'.format(topic=topic, q=q, resp=resp)
            discriminator_q += '\nКак ты считаешь, является ли предложенный ответ на вопрос правильным или нет? \
                Оцени вероятность правильности ответа по шкале от 0 до 10, где 0 - точно неправильно и 10 - точно правильно.\
                В ответ пиши только одну цифру.'
            discriminator_resp = await async_ask(discriminator_q, top_k=1)
            try:
                rating = int(''.join(c for c in discriminator_resp if c in '0123456789'))
                answers.append((resp, rating))
                if verbose:
                    print(f'<Descriminator rating>:\n{resp}')
                if rating >= threshold:
                    break
            except:
                print(f'Not a number was returned as a rating value: {discriminator_resp}')
        
        if answers[-1][1] < threshold:
            max_rating = -np.inf
            for answ, rating in answers:
                if rating > max_rating:
                    max_rating = rating
                    best_answ = answ
        else:
            best_answ = answers[-1][0]
        
        envoy_q = 'Ты - участник команды в игре "Что? Где? Когда?". Тебе дан вопрос, тема вопроса и ответ капитана команды.\
            \nТема вопроса: {topic}\nВопрос: {q}\nОтвет капитана команды {best_answ}'.format(topic=topic, q=q, best_answ=best_answ)
        envoy_q += '\nТвоя задача сформировать окончательный ответ команды в соответствии с ответом капитана.\
            Обязательно пиши ответ только на русском языке. Ответ должен быть кратким, без пояснений. \
            Не добавляй никаких дополнительных знаков пунктуации в начале и в конце.\nОтвет: '
        envoy_resp = await async_ask(envoy_q, top_k=1, clear=True)
        
        if logId is not None:
            logDir = f'/home/jupyter/datasphere/project/log/{logId}'
            os.makedirs(logDir, exist_ok=True)
            with open(f'{logDir}/q{row_idx}.txt', 'w') as fwrite:
                fwrite.write(f'<Qestion>: {q}\n')
                for i, (resp, rating) in enumerate(answers):
                    fwrite.write(f'\n<Response {i}>: {resp}')
                    fwrite.write(f'\n<Rating {i}>: {rating}')
                fwrite.write(f'<Final answer>: {envoy_resp}')

        return envoy_resp
    
    return ask_func

In [54]:
async_ask_sd_9_7 = async_get_suggest_discrim(threshold=9)

In [45]:
await async_ask_sd_9_7(df_to_eval, 0, verbose=True, logId='SuggesterDiscriminator-9-7')

<Развернутый ответ>:
Чтобы ответить на этот вопрос нам нужно проанализировать тему Бэби и понять как она связана со средневековыми аналогами и вращающейся дверью

Шаг 1 Анализ темы Бэби
Тема Бэби обычно ассоциируется с младенцами или маленькими детьми Однако учитывая контекст вопроса мы можем предположить что Бэби может быть также сокращением или прозвищем для чегото другого

Шаг 2 Средневековые аналоги
Средневековые аналоги подразумевают существование чегото в Средние века которое имеет сходство или аналогичные черты с предметом вопроса

Шаг 3 Вращающаяся дверь
Вращающаяся дверь  это ключевой элемент конструкции упомянутый в вопросе Вращающиеся двери часто используются в архитектуре для создания удобного входа и выхода

Шаг 4 Связь между Бэби и средневековыми аналогами с вращающейся дверью
После анализа темы и элементов вопроса можно сделать предположение что Бэби может быть прозвищем или сокращением для некоторого типа конструкции или устройства которое использовалось в Средние века 

'Бэбибокс'

Started 10/8 ?; finished 10/8 ?

In [55]:
f1_score, em_score, responses_, f1_scores = await async_inference(df_to_eval.iloc[:, :], async_ask_sd_9_7, logId='SuggesterDiscriminator-9-7')


Prediction timed out.
f1 0.48992120190164695; EM: {'exact_match': 0.23557692307692307}


## Suggester Discriminator with Critical Analysis SDwCA

In [56]:
def async_get_SDwCA(threshold=8, max_attempts=7, max_CoT_steps=4):
    '''
    Scheme:
        q -> suggester -> answ
        answ -> discriminator -> rating
        if rating >= threshold
            return asnw
        answ, rating -> critic -> critique
        answs, critic -> summer -> summary
        summary -> suggester -> new_answ
        repeat the cycle
    '''
    
    async def ask_func(df_, row_idx, verbose=False, logId=None):    
        topic = df_.inputs[row_idx].get('topic', '')
        q = df_.inputs[row_idx]['text']
        
        outputs = []
        summaries = []
        
        for i in range(max_attempts):
            curr_output = []
            if len(outputs) == 0:
                suggester_q = 'Ты - участник игры "Что? Где? Когда?", и твоя задача ответить на заданный тебе вопрос по заданной теме.\
                    \nТема вопроса: {topic}\nВопрос: {q}'.format(topic=topic, q=q)
                suggester_q += '\nОбоснуй свой ответ. Рассуждай шаг за шагом,\
                    при этом в рассуждении должно быть не более чем {limit} шагов.'.format(limit=max_CoT_steps)
            else:
                summer_q = 'Ты - участник команды в игре "Что? Где? Когда?". Тебе известны вопрос и тема вопроса.\
                    \nТема вопроса: {topic}\nВопрос: {q}'.format(topic=topic, q=q)
                summer_q += '\n\nРанее другие участники уже пытались ответить на заданный вопрос, но их ответы были неудовлетворительны.'
                for j, (answ, r, critic) in enumerate(outputs):
                    summer_q += f'\nОтвет {j}й: {answ}\nКритика {j}го ответа: {critic}\nОценка {j}го ответа: {r}'                
                summer_q += '\n\nТвоя задача кратко суммировать все ответы и критику на них.'
                summer_resp = await async_ask(summer_q)
                summaries.append(summer_resp)
                
                suggester_q = 'Ты - участник команды в игре "Что? Где? Когда?", и твоя задача ответить на заданный тебе вопрос по заданной теме.\
                    \nТема вопроса: {topic}\nВопрос: {q}'.format(topic=topic, q=q)
                
                ## all answers, critiques approach
                # suggester_q += 'Ранее другие участники уже пытались ответить на заданный вопрос, но их ответы были неудовлетворительны.'
                # for j, (answ, r, critic) in enumerate(outputs):
                #     suggester_q += f'\nОтвет {j}й: {answ}\nКритика {j}го ответа: {critic}\nОценка {j}го ответа: {r}'
                # suggester_q += '\nТвоя задача предложить свой вариант ответа на вопрос, учитывая критику вариантов других участников.\
                #     Обоснуй свой ответ. Рассуждай шаг за шагом.'
                
                ## last answer, critic approach
                # suggester_q += '\nРанее другой участник уже пытался ответить на заданный вопрос, но его ответ был неудовлетворителен.'
                # answ, critic, r = outputs[-1]
                # suggester_q += f'\nОтвет другого участника: {answ}\
                #     \nКритика ответа другого участника: {critic}\
                #     \nОценка ответа  другого участника: {r}'
                # suggester_q += '\nТвоя задача предложить свой вариант ответа на вопрос, учитывая критику варианта другого участника.\
                #     Обоснуй свой ответ. Рассуждай шаг за шагом.'
                
                # summary approach
                suggester_q += 'Ранее другие участники уже пытались ответить на заданный вопрос, но их ответы были неудовлетворительны.\
                    Ответы и критика на них в краткой форме приведены ниже:\n{summary}'.format(summary=summer_resp)
                suggester_q += '\n\nТвоя задача предложить свой вариант ответа на вопрос, учитывая критику вариантов других участников.\
                    Обоснуй свой ответ. Рассуждай шаг за шагом,\
                    при этом в рассуждении должно быть не более чем {limit} шагов.'.format(limit=max_CoT_steps)
            resp = await async_ask(suggester_q)
            if verbose:
                print(f'<Развернутый ответ>:\n{resp}')

            discriminator_q = 'Ты - участник команды в игре "Что? Где? Когда?". Тебе дан вопрос, тема вопроса и ответ другого участника.\
                \nТема вопроса: {topic}\nВопрос: {q}\nОтвет другого участника {resp}'.format(topic=topic, q=q, resp=resp)
            discriminator_q += '\nКак ты считаешь, является ли предложенный ответ на вопрос правильным или нет? \
                Оцени вероятность правильности ответа по шкале от 0 до 10, где 0 - точно неправильно и 10 - точно правильно.\
                В ответ пиши только одну цифру.'
            discriminator_resp = await async_ask(discriminator_q, top_k=1)
            try:
                rating = int(''.join(c for c in discriminator_resp if c in '0123456789'))
                curr_output.extend([resp, rating])
                if verbose:
                    print(f'<Descriminator rating>:\n{rating}')
                if rating >= threshold:
                    curr_output.append('Good answer reached')
                    outputs.append(tuple(curr_output))
                    break
            except ValueError:
                print(f'Not correct value for rating was returned: {discriminator_resp}')
                continue
            critic_q = 'Ты - участник команды в игре "Что? Где? Когда?". Тебе дан вопрос, тема вопроса, ответ на вопрос,\
                предложенный другим участником команды, и оценка этого ответа капитаном команды по шкале от 0 до 10.\
                \nТема вопроса: {topic} \nВопрос: {q}\nОтвет другого участника {resp}\nОценка ответа: {rating}\
                Предложенный участником ответ был отвергнут капитаном как неудовлетворительный.\
                \nТвоя задача объяснить почему предложенный ответ был признан неудовлетворительным.\
                \nКритика ответа: '.format(topic=topic, q=q, resp=resp, rating=rating)
            critic_resp = await async_ask(critic_q)
            if verbose:
                print(f'<Критика>:\n{critic_resp}')
            curr_output.append(critic_resp)
            outputs.append(tuple(curr_output))
        
        if outputs[-1][1] < threshold:
            max_rating = -np.inf
            for answ, rating, _ in outputs:
                if rating > max_rating:
                    max_rating = rating
                    best_answ = answ
        else:
            best_answ = outputs[-1][0]
        
        envoy_q = 'Ты - участник команды в игре "Что? Где? Когда?". Тебе дан вопрос, тема вопроса и ответ капитана команды.\
            \nТема вопроса: {topic}\nВопрос: {q}\nОтвет капитана команды {best_answ}'.format(topic=topic, q=q, best_answ=best_answ)
        envoy_q += '\nТвоя задача сформировать окончательный ответ команды в соответствии с ответом капитана.\
            Обязательно пиши ответ только на русском языке. Ответ должен быть кратким, без пояснений. \
            Не добавляй никаких дополнительных знаков пунктуации в начале и в конце.\nОтвет: '
        envoy_resp = await async_ask(envoy_q, top_k=1, clear=True)
        
        if logId is not None:
            logDir = f'/home/jupyter/datasphere/project/log/{logId}'
            os.makedirs(logDir, exist_ok=True)
            with open(f'{logDir}/q{row_idx}.txt', 'w') as fwrite:
                fwrite.write(f'<Qestion>: {q}\n')
                for i, (resp, rating, critique) in enumerate(outputs):
                    if i > 0:
                        fwrite.write(f'\n<Summary {i-1}>: {summaries[i-1]}\n')
                    fwrite.write(f'\n<Response {i}>: {resp}\n')
                    fwrite.write(f'\n<Rating {i}>: {rating}\n')
                    fwrite.write(f'\n<Critque {i}>: {critique}\n')
                fwrite.write(f'\n<Final answer>: {envoy_resp}\n')

        return envoy_resp
    
    return ask_func

In [57]:
async_ask_SDwCA_9_7 = async_get_SDwCA(threshold=9)

In [59]:
await async_ask_SDwCA_9_7(df_to_eval, 2, verbose=True, logId='SDwCA-9-7')

<Развернутый ответ>:
Отлично, я готов ответить на вопрос. Вот мои рассуждения:

Шаг 1: Нам дана тема "Гласные - только 'А'", что означает, что ответ должен содержать только гласную букву "А".

Шаг 2: Вопрос упоминает Державина, известного русского поэта, и сравнение богатого и ленивого вельможи с 33-м потомком Нина.

Шаг 3: Нин - библейский персонаж, известный как правнук Ноя. Если мы посчитаем 33 поколения от Нина, мы можем прийти к известному историческому или литературному персонажу.

Шаг 4: Учитывая, что Державин сравнивает богатого и ленивого вельможу с этим человеком, я предполагаю, что ответом может быть "Хан", который является 33-м потомком Нина и известен своей роскошной и ленивой жизнью.

Ответ: Хан.
<Descriminator rating>:
8
<Критика>:
Ответ другого участника команды был отвергнут капитаном, потому что он не соответствует теме "Гласные - только 'А'". В слове "Хан" присутствует гласная буква "А", но в имени, которое Державин использовал для сравнения, должно быть только гласн

'Хан'

In [60]:
f1_score, em_score, responses_, f1_scores = await async_inference(df_to_eval.iloc[:, :], async_ask_SDwCA_9_7, logId='SDwCA-9-7')

Not correct value for rating was returned: Извините, но я не могу предоставлять контент, связанный с взрослыми темами.
f1 0.4804878127188851; EM: {'exact_match': 0.23317307692307693}
